# Driver Drowsiness Detection

## Importing Libraries

In [11]:
import cv2 as cv
from keras.models import model_from_json
import tensorflow as tf
import numpy as np
from keras.preprocessing import image

## Loading CNN Model

In [12]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


### Face and Eye Detection using Haar Cascade Detection

In [23]:
face_cascade_name = "haarcascade_frontalface_default.xml"
eyes_cascade_name = "haarcascade_eye_tree_eyeglasses.xml"
l_eye_cascade_name = "haarcascade_lefteye_2splits.xml"
r_eye_cascade_name = "haarcascade_righteye_2splits.xml"

face_cascade = cv.CascadeClassifier(face_cascade_name)
eye_cascade = cv.CascadeClassifier(eyes_cascade_name)
l_eye_cascade = cv.CascadeClassifier(l_eye_cascade_name)
r_eye_cascade = cv.CascadeClassifier(r_eye_cascade_name)
#Works with gray scale images. So converting into GrayScale images

font = cv.FONT_HERSHEY_COMPLEX_SMALL

cap = cv.VideoCapture(0)

while cap.isOpened():
    _,  frame = cap.read()
    
    gray = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,1.1,4)

    for (x,y,w,h) in faces:
        roi_color = frame[y:y+h, x:x+w]
        roi_gray = gray[y:y+h, x:x+w]
#         eyes = eye_cascade.detectMultiScale(roi_gray)
        l_eye = l_eye_cascade.detectMultiScale(roi_gray)
        r_eye = r_eye_cascade.detectMultiScale(roi_gray)
        cv.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 3)
        
        for (ex,ey,ew,eh) in l_eye:
            #Prediction
            l_eye_frame = frame[ey:ey+eh,ex:ex+ew]
            l_eye_frame = cv.resize(l_eye_frame,(64,64))
            l_eye_frame = l_eye_frame.reshape(64,64,-1)
            l_eye_frame = image.img_to_array(l_eye_frame)
            l_eye_frame = np.expand_dims(l_eye_frame, axis = 0)
            result = model.predict(l_eye_frame)
                      
            cv.rectangle(roi_color, (ex,ey), (ex+ew, ey+eh), (0,255,0), 2)
            
            if result[0][0] == 0:
                cv.putText(frame,"Closed",(10,h-20), font, 1,(255,255,255),1,cv.LINE_AA)
                print("Closed")
            else:
                cv.putText(frame,"Open",(10,h-20), font, 1,(255,255,255),1,cv.LINE_AA)
                print("Open")
            
        for (ex,ey,ew,eh) in r_eye:
            #Prediction
            r_eye_frame = frame[ey:ey+eh,ex:ex+ew]
            r_eye_frame = cv.resize(r_eye_frame,(64,64))
            r_eye_frame = r_eye_frame.reshape(64,64,-1)
            r_eye_frame = image.img_to_array(r_eye_frame)
            r_eye_frame = np.expand_dims(r_eye_frame, axis = 0)
            result = model.predict(r_eye_frame)
            
            cv.rectangle(roi_color, (ex,ey), (ex+ew, ey+eh), (0,255,0), 2)
            
            if result[0][0] == 0:
                cv.putText(frame,"Closed",(w-20,h-20), font, 1,(255,255,255),1,cv.LINE_AA)
            else:
                cv.putText(frame,"Open",(w-20,h-20), font, 1,(255,255,255),1,cv.LINE_AA)
            
            
            
            
    cv.imshow('img',frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        cv.destroyAllWindows()
        cap.release()
        break

Open
Open
Open
Open
Open
Open
Open
Closed
Closed
Open
Closed
Closed
Open
Open
Open
Closed
Open
Closed
Open
Open
Open
Closed
Open
Open
Closed
Open
Closed
Closed
Open
Closed
Open
Closed
Open
Closed
Open
Closed
Open
Closed
Open
Closed
Open
Closed
Open
Closed
Open
Closed
Open
Open
Closed
Closed
Open
Closed
Open
Closed
Open
Closed
Open
Open
Closed
Closed
Open
Open
Closed
Open
Closed
Closed
Open
Open
Closed
Closed
Open
Closed
Open
Closed
Open
Closed
Open
Closed
Open
Closed
Open
Closed
Closed
Closed
Open
Closed
Closed
Closed
Closed
Closed
Open
Closed
Open
Open
Closed
Closed
Open
Closed
Open
Closed
Open
Open
Open
Closed
Closed
Closed
Open
Closed
Open
Closed
Closed
Open
Closed
Closed
Open
Closed
Closed
Closed
Closed
Closed
Closed
Closed
Open
Open
Closed
Closed
Closed
Open
Closed
Closed
Closed
Closed
Open
Closed
Closed
Closed
Closed
Closed
Closed
Closed
Closed
Open
Closed
Closed
Closed
Closed
Closed
Closed
Closed
Closed
Closed
Closed
Closed
Closed
Closed
Closed
Closed
Closed
Open
Closed
Closed
C

### Driver Drowsiness Detection using Pretrained model

In [29]:
import cv2
import os
from keras.models import load_model
import numpy as np
# from pygame import mixer
import time


# mixer.init()
# sound = mixer.Sound('alarm.wav')

face = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_righteye_2splits.xml')



lbl=['Close','Open']

model = load_model('models/cnncat2.h5')
path = os.getcwd()
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
thicc=2
rpred=[99]
lpred=[99]

while(True):
    _, frame = cap.read()
    height,width = frame.shape[:2] 

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    left_eye = leye.detectMultiScale(gray)
    right_eye =  reye.detectMultiScale(gray)

    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )

    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )

    for (x,y,w,h) in right_eye:
        r_eye=frame[y:y+h,x:x+w]
        count=count+1
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye,(24,24))
        r_eye= r_eye/255
        r_eye=  r_eye.reshape(24,24,-1)
        r_eye = np.expand_dims(r_eye,axis=0)
        rpred = model.predict(r_eye)
        if(rpred[0][0]==1):
            lbl='Open' 
        if(rpred[0][0]==0):
            lbl='Closed'
        break

    for (x,y,w,h) in left_eye:
        l_eye=frame[y:y+h,x:x+w]
        count=count+1
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)  
        l_eye = cv2.resize(l_eye,(24,24))
        l_eye= l_eye/255
        l_eye=l_eye.reshape(24,24,-1)
        l_eye = np.expand_dims(l_eye,axis=0)
        lpred = model.predict(l_eye)
        if(lpred[0][0]==1):
            lbl='Open'   
        if(lpred[0][0]==0):
            lbl='Closed'
        break

    if(rpred[0][0]==0 and lpred[0][0]==0):
        score=score+1
        cv2.putText(frame,"Closed",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    # if(rpred[0]==1 or lpred[0]==1):
    else:
        score=score-1
        cv2.putText(frame,"Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    
        
    if(score<0):
        score=0   
    cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    if(score>15):
        #person is feeling sleepy so we beep the alarm
        cv2.imwrite(os.path.join(path,'image.jpg'),frame)
#         try:
#             sound.play()
            
#         except:  # isplaying = False
#             pass
        if(thicc<16):
            thicc= thicc+2
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc) 
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
